All text describing the algorithms were taken from the api reference of Sktime.   
https://www.sktime.org/en/stable/api_reference/classification.html# 

# Required Imports

In [1]:
import pandas as pd 
import numpy as np 
from sklearn.metrics import accuracy_score

## Dataset
Dimensionality: univariate Series length: 251 Train cases: 36 Test cases: 175 Number of classes: 3

The arrowhead data consists of outlines of the images of arrowheads. The shapes of the projectile points are converted into a time series using the angle-based method. The classification of projectile points is an important topic in anthropology. The classes are based on shape distinctions such as the presence and location of a notch in the arrow. The problem in the repository is a length normalised version of that used in Ye09shapelets. The three classes are called “Avonlea”, “Clovis” and “Mix”.”

Dataset details: http://timeseriesclassification.com/description.php?Dataset=ArrowHead

In [2]:
from sktime.datasets import load_gunpoint
X_train, y_train = load_gunpoint(split="train", return_X_y=True)
X_test, y_test = load_gunpoint(split="test", return_X_y=True)

C:\Users\edwar\anaconda3\envs\tsc\lib\site-packages\sktime\datatypes\_series\_check.py:43: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_INDEX_TYPES = (pd.Int64Index, pd.RangeIndex, pd.PeriodIndex, pd.DatetimeIndex)
C:\Users\edwar\anaconda3\envs\tsc\lib\site-packages\sktime\datatypes\_hierarchical\_check.py:50: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_INDEX_TYPES = (pd.Int64Index, pd.RangeIndex, pd.PeriodIndex, pd.DatetimeIndex)
C:\Users\edwar\anaconda3\envs\tsc\lib\site-packages\sktime\datatypes\_hierarchical\_check.py:51: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_MULTIINDEX_TYPES = (pd.Int64Index, pd.RangeIndex)
C:\Users\edwar\anaconda3

# Dictionary Based

## IndividualBOSS 

Single bag of Symbolic Fourier Approximation Symbols (IndividualBOSS).

Bag of SFA Symbols Ensemble: implementation of a single BOSS Schaffer, the base classifier for the boss ensemble.

Implementation of single BOSS model from Schäfer (2015)

This is the underlying classifier for each classifier in the BOSS ensemble.

Overview: input “n” series of length “m” and IndividualBoss performs a SFA transform to form a sparse dictionary of discretised words. The resulting dictionary is used with the BOSS distance function in a 1-nearest neighbor.

Fit involves finding “n” histograms.

Predict uses 1 nearest neighbor with a bespoke BOSS distance function.

In [3]:
from sktime.classification.dictionary_based import IndividualBOSS
clf = IndividualBOSS()
clf.fit(X_train, y_train)

IndividualBOSS()

In [4]:
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9266666666666666

## BOSSEnsemble (BOSS)

Ensemble of bag of Symbolic Fourier Approximation Symbols (BOSS).

Implementation of BOSS Ensemble from Schäfer (2015). [1]

Overview: Input “n” series of length “m” and BOSS performs a grid search over a set of parameter values, evaluating each with a LOOCV. It then retains all ensemble members within 92% of the best by default for use in the ensemble. There are three primary parameters:

- alpha: alphabet size
- w: window length
- l: word length.

For any combination, a single BOSS slides a window length “w” along the series. The w length window is shortened to an “l” length word through taking a Fourier transform and keeping the first l/2 complex coefficients. These “l” coefficients are then discretized into alpha possible values, to form a word length “l”. A histogram of words for each series is formed and stored.

Fit involves finding “n” histograms.

Predict uses 1 nearest neighbor with a bespoke BOSS distance function.

In [5]:
from sktime.classification.dictionary_based import BOSSEnsemble
clf = BOSSEnsemble(max_ensemble_size=5)
clf.fit(X_train, y_train)

BOSSEnsemble(max_ensemble_size=5)

In [6]:
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

1.0

## ContractableBOSS (CBOSS)

Contractable Bag of Symbolic Fourier Approximation Symbols (cBOSS).

Implementation of BOSS Ensemble from Schäfer (2015) with refinements described in Middlehurst, Vickers and Bagnall (2019). [1, 2]_

Overview: Input “n” series of length “m” and cBOSS randomly samples n_parameter_samples parameter sets, evaluting each with LOOCV. It then retains max_ensemble_size classifiers with the highest accuracy There are three primary parameters:

- alpha: alphabet size
- w: window length
- l: word length.

For any combination, a single BOSS slides a window length “w” along the series. The “w” length window is shortened to an “l” length word by taking a Fourier transform and keeping the first l/2 complex coefficients. These “l” coefficients are then discretised into “alpha” possible values, to form a word length “l”. A histogram of words for each series is formed and stored.

Fit involves finding “n” histograms.

Predict uses 1 nearest neighbor with a bespoke BOSS distance function.

In [7]:
from sktime.classification.dictionary_based import ContractableBOSS
clf = ContractableBOSS(n_parameter_samples=25, max_ensemble_size=5)
clf.fit(X_train, y_train)

ContractableBOSS(max_ensemble_size=5, n_parameter_samples=25)

In [8]:
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9933333333333333

# Distance Based


## KNN Time Series Classifier.



KNN Time Series Classifier.

An adapted version of the scikit-learn KNeighborsClassifier to work with time series data.

Necessary changes required for time series data:
- calls to X.shape in kneighbors, predict and predict_proba. In the base class, these methods contain:

```python
n_samples, _ = X.shape
```

This however assumes that data must be 2d (a set of multivariate time series is 3d). Therefore these methods needed to be overridden to change this call to the following to support 3d data:
```python
n_samples = X.shape[0]
```
- check array has been disabled. This method allows nd data via an argument in the method header. However, there
seems to be no way to set this in the classifier and allow it to propagate down to the method. Therefore, this method has been temporarily disabled (and then re-enabled). It is unclear how to fix this issue without either writing a new classifier from scratch or changing the scikit-learn implementation. TO-DO: find permanent resolution to this issue (raise as an issue on sklearn GitHub?)

In [9]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
classifier = KNeighborsTimeSeriesClassifier()
classifier.fit(X_train, y_train)

KNeighborsTimeSeriesClassifier()

In [10]:
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9933333333333333

## Elastic Ensemble (EE)

The Elastic Ensemble (EE).

EE as described in [1].

Overview:
Input n series length m
EE is an ensemble of elastic nearest neighbor classifiers

In [11]:
from sktime.classification.distance_based import ElasticEnsemble
clf = ElasticEnsemble(
    proportion_of_param_options=0.1,
    proportion_train_for_test=0.1,
)
clf.fit(X_train, y_train)

ElasticEnsemble(distance_measures=['dtw', 'ddtw', 'wdtw', 'wddtw', 'lcss',
                                   'erp', 'msm'],
                proportion_of_param_options=0.1, proportion_train_for_test=0.1)

In [12]:
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9866666666666667

## Proximity Forest (PF)

Proximity Forest class.

Models a decision tree forest which uses distance measures to partition data [1].

In [13]:
from sktime.classification.distance_based import ProximityForest
clf = ProximityForest(n_estimators=5)
clf.fit(X_train, y_train)

ProximityForest(distance_measure=<function distance_predefined_params.<locals>.distance at 0x0000013D444AE040>,
                get_distance_measure=<function setup_all_distance_measure_getter.<locals>.pick_rand_distance_measure at 0x0000013D7D53F790>,
                n_estimators=5,
                random_state=RandomState(MT19937) at 0x13D76904040)

In [14]:
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.96

# Hybrid

## HIVECOTEV2

Hierarchical Vote Collective of Transformation-based Ensembles (HIVE-COTE) V2.

An ensemble of the STC, DrCIF, Arsenal and TDE classifiers from different feature representations using the CAWPE structure as described in [1].

In [15]:
from sktime.classification.hybrid import HIVECOTEV2
from sktime.contrib.vector_classifiers._rotation_forest import RotationForest
clf = HIVECOTEV2(
    stc_params={
        "estimator": RotationForest(n_estimators=3),
        "n_shapelet_samples": 500,
        "max_shapelets": 20,
        "batch_size": 100,
    },
    drcif_params={"n_estimators": 10},
    arsenal_params={"num_kernels": 100, "n_estimators": 5},
    tde_params={
        "n_parameter_samples": 25,
        "max_ensemble_size": 5,
        "randomly_selected_params": 10,
    },
)
clf.fit(X_train, y_train)

HIVECOTEV2(arsenal_params={'n_estimators': 5, 'num_kernels': 100},
           drcif_params={'n_estimators': 10},
           stc_params={'batch_size': 100,
                       'estimator': RotationForest(n_estimators=3),
                       'max_shapelets': 20, 'n_shapelet_samples': 500},
           tde_params={'max_ensemble_size': 5, 'n_parameter_samples': 25,
                       'randomly_selected_params': 10})

In [16]:
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

1.0

# Interval Based

## Time Series Forest (TSF)

Time series forest classifier.

A time series forest is an ensemble of decision trees built on random intervals. Overview: Input n series length m. For each tree

- sample sqrt(m) intervals,
- find mean, std and slope for each interval, concatenate to form newdata set, 
- build decision tree on new data set.

Ensemble the trees with averaged probability estimates.

This implementation deviates from the original in minor ways. It samples intervals with replacement and does not use the splitting criteria tiny refinement described in [1].

This is an intentionally stripped down, non configurable version for use as a hive-cote component. For a configurable tree based ensemble, see sktime.classifiers.ensemble.TimeSeriesForestClassifier

In [17]:
from sktime.classification.interval_based import TimeSeriesForestClassifier
clf = TimeSeriesForestClassifier(n_estimators=10)
clf.fit(X_train, y_train)

TimeSeriesForestClassifier(n_estimators=10)

In [18]:
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.96

## RandomIntervalSpectralEnsemble (RISE)

Random Interval Spectral Ensemble (RISE).

Input: n series length m For each tree

- sample a random intervals
- take the ACF and PS over this interval, and concatenate features
- build tree on new features

Ensemble the trees through averaging probabilities.

In [19]:
from sktime.classification.interval_based import RandomIntervalSpectralEnsemble
clf = RandomIntervalSpectralEnsemble(n_estimators=10)
clf.fit(X_train, y_train)

RandomIntervalSpectralEnsemble(n_estimators=10)

In [20]:
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.98

# Shaplet

## ShapletTransform

Shapelet Transform Classifier.

Implementation of the binary shapelet transform classifier along the lines of [Rdd57c5f36417-1]_[Rdd57c5f36417-2]_. Transforms the data using the configurable shapelet transform and then builds a rotation forest classifier. As some implementations and applications contract the classifier solely, contracting is available for the transform only and both classifier and transform.

In [21]:
from sktime.classification.shapelet_based import ShapeletTransformClassifier
from sktime.contrib.vector_classifiers._rotation_forest import RotationForest
clf = ShapeletTransformClassifier(
    estimator=RotationForest(n_estimators=3),
    n_shapelet_samples=500,
    max_shapelets=20,
    batch_size=100,
)
clf.fit(X_train, y_train)

ShapeletTransformClassifier(estimator=RotationForest(n_estimators=3),
                            max_shapelets=20, n_shapelet_samples=500)

In [22]:
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9733333333333334